In [1]:
import sys
sys.path.append("../")

In [2]:
import glob

path_dir_test_data = "../test_data/html"
list_path_html = glob.glob(path_dir_test_data + "/*.html")
len(list_path_html)

10

In [3]:
from langchain.document_loaders import BSHTMLLoader, UnstructuredHTMLLoader

loader = BSHTMLLoader(list_path_html[0], "utf-8")
loaded = loader.load()  # -> List[Document]

In [4]:
type(loaded[0].page_content)

str

In [5]:
from docmesh.embedding import LangchainOpenAIEmbeddingModel
from docmesh.llm import LangchainOpenAILLM
from docmesh.config import Config

from dotenv import load_dotenv

load_dotenv()


generator_llm = LangchainOpenAILLM(model=Config.LLM_MODEL)
critic_llm = LangchainOpenAILLM(model=Config.LLM_MODEL)
embedding = LangchainOpenAIEmbeddingModel()

In [6]:
from docmesh.evaluation import RAGTestsetGenerator, load_personas_from_yaml

list_persona = load_personas_from_yaml('persona.yaml')
list_persona

[Persona(name='Junior Developer', role_description='A developer who has just started their career in software development. They have basic programming knowledge and are actively learning how to collaborate on small tasks or bug fixes.'),
 Persona(name='Intermediate Developer', role_description='A developer with around 2–5 years of professional experience. They can independently handle most tasks using common frameworks and libraries, have a reasonable grasp of code quality and design patterns, and can mentor junior developers.'),
 Persona(name='Senior Developer', role_description='A seasoned developer with 5 or more years of hands-on experience. They are proficient in large-scale system design, performance optimization, and can effectively lead teams or mentor other developers.'),
 Persona(name='University Student', role_description='A student majoring in computer science or a related field, learning fundamental programming skills and theoretical knowledge. They may lack practical expe

In [7]:
generator = RAGTestsetGenerator(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    generator_embeddings=embedding,
    list_persona=list_persona
                               )

In [8]:
from ragas.testset.transforms import HeadlineSplitter, HeadlinesExtractor
from ragas.testset.transforms.extractors import NERExtractor


transforms = [
    HeadlinesExtractor(), 
    HeadlineSplitter(), 
    NERExtractor()
]

In [ ]:
result = generator.create_testset(list_path_html, testset_size=len(list_path_html), transforms=transforms)
result_pandas = result.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
result_pandas

In [ ]:
result_pandas.to_csv("ragas_synthetic_dataset.csv", index=False)